# Plotly visualization

In [ ]:
import textwrap

def customwrap(s, width=38):
    return "<br>".join(textwrap.wrap(s, width=width))

## 1. Bar chart
Bar chart of the number of recipes per category or subcategory

In [ ]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df = pd.read_json("data/structured_data.json", orient="table")
df.head()

In [ ]:
# Wrap long subcategory names
df["subcategory"] = df["subcategory"].apply(customwrap)

In [ ]:
# Bar chart using plotly
recipes_count_per_category = df.groupby("category").count()["recipe_name"].reset_index()
recipes_count_per_category.columns = ["Категория", "Число рецептов"]

# Sort by number of recipes
recipes_count_per_category = recipes_count_per_category.sort_values(by="Число рецептов", ascending=False)
recipes_count_per_category

In [ ]:
# Disable x-axis labels
fig = px.bar(
    recipes_count_per_category,
    x="Категория",
    y="Число рецептов",
    title="Число рецептов по категориям",
    color="Категория",
)
fig.update_xaxes(showticklabels=False, title=None)
fig.update_layout(
    paper_bgcolor="rgba(0, 0, 0, 0)",
    plot_bgcolor="rgba(0, 0, 0, 0)",
    font_family="Arial Black",
    title_font_family="Arial Black",
)
fig.show()

# Save figure
os.makedirs("visualization/images", exist_ok=True)
fig.write_image("visualization/images/recipes_count_per_category.svg")

In [ ]:
## For each category show a bar chart with the number of recipes per subcategory
recipes_count_per_subcategory = df.groupby(["category", "subcategory"]).count()["recipe_name"].reset_index()
recipes_count_per_subcategory.columns = ["Категория", "Подкатегория", "Число рецептов"]

# Sort by count per each category
recipes_count_per_subcategory = recipes_count_per_subcategory.sort_values(
    by=["Категория", "Число рецептов"], ascending=[True, False]
)
recipes_count_per_subcategory

In [ ]:
for category in recipes_count_per_subcategory["Категория"].unique():
    fig = px.bar(
        recipes_count_per_subcategory[recipes_count_per_subcategory["Категория"] == category],
        x="Подкатегория",
        y="Число рецептов",
        color="Подкатегория",
        title=f"Число рецептов по подкатегориям в категории {category}",
        text_auto=True,
        width=960,
    )
    fig.update_xaxes(showticklabels=False, title=None)
    fig.update_layout(
        paper_bgcolor="rgba(0, 0, 0, 0)",
        plot_bgcolor="rgba(0, 0, 0, 0)",
        font_family="Arial Black",
        title_font_family="Arial Black",
        legend = dict(
            entrywidth=0.45,
            entrywidthmode='fraction',
            orientation="h",
            xanchor="center",
            x=0.5,
            yanchor="top",
            y=-0.15,
            title=None,
        ),
    )
    fig.show()

    # Save figure
    os.makedirs("visualization/images/stats_per_category", exist_ok=True)
    fig.write_image(f"visualization/images/stats_per_category/{category}.svg")

## 2. Word cloud
Word cloud of the most commonly used ingredients

In [ ]:
import pandas as pd

from collections import defaultdict

In [ ]:
df = pd.read_json("data/structured_data.json", orient="table")
df.head()

In [ ]:
# Word cloud
ingredients_list_counts = defaultdict(int)

for ingredients in df["parsed_ingredients"]:
    for ingredient in ingredients:
        if ingredient:
            ingredients_list_counts[ingredient[0]] += 1

# Sort by count
ingredients_list_counts = dict(sorted(ingredients_list_counts.items(), key=lambda x: x[1], reverse=True))

# Save to file
with open("visualization/data/ingredients.txt", "w", encoding="utf-8") as f:
    for ingredient, count in ingredients_list_counts.items():
        if ingredient is not None:
            f.write(f"{ingredient} {count}\n")

In [ ]:
# Show svg file
from IPython.display import SVG, display

display(SVG(filename="visualization/images/wordcloud.svg"))

## 3. Sunburst chart

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df = pd.read_json("data/structured_data.json", orient="table")
df.head()

In [ ]:
# Wrap long subcategory names
df["subcategory"] = df["subcategory"].apply(customwrap, width=20)

In [ ]:
categories_names = df.groupby("category").count().reset_index()["category"].tolist()
categories_values = df.groupby("category").count().reset_index()["recipe_name"].tolist()

subcategories_names = df.groupby(["category", "subcategory"]).count().reset_index()["subcategory"].tolist()
subcategories_parents = df.groupby(["category", "subcategory"]).count().reset_index()["category"].tolist()
subcategories_values = df.groupby(["category", "subcategory"]).count().reset_index()["recipe_name"].tolist()

trace = go.Sunburst(
    labels=["БЛЮДА"] + categories_names + subcategories_names,
    parents=[""] + ["БЛЮДА"] * len(categories_names) + subcategories_parents,
    values=[df.shape[0]] + categories_values + subcategories_values,
    branchvalues="total",
    marker={"line": {"width": 2}},
    textinfo="label+percent entry",
    hovertemplate="<b>Рецептов: %{value}</b><extra></extra>",
    insidetextorientation='radial'
)

layout = go.Layout(
    paper_bgcolor="rgba(0, 0, 0, 0)",
    plot_bgcolor="rgba(0, 0, 0, 0)",
    margin = go.layout.Margin(t=0, l=0, r=0, b=0),
    width=900,
    height=900,
    font_family="Arial Black"
)

figure = {
    "data": [trace],
    "layout": layout
}

fig = go.Figure(figure)
fig.show()

# Save figure
with open("visualization/images/sunburst.html", "w", encoding="utf-8") as f:
    f.write(fig.to_html(full_html=False, include_plotlyjs="cdn"))

## 4. Interactive Network Visualization

In [ ]:
import json
import os
import pandas as pd

In [ ]:
df = pd.read_json("data/structured_data.json", orient="table")
df.head()

In [ ]:
recipes_ingredients = []

for category, subcategory, recipe_name, ingredients in zip(df["category"], df["subcategory"], df["recipe_name"], df["parsed_ingredients"]):
    recipes_ingredients.append(
        {
            "category": category,
            "subcategory": subcategory,
            "recipe_name": recipe_name,
            "ingredients": [
                ingredient[0] for ingredient in ingredients if ingredient[0]
            ]
        }
    )

recipes_ingredients

### We need to gather data in the following format:
```json
{
  "nodes": [
    {
      "is_recipe": true,
      "recipe_name": "recipe name",
      "id": "unique_id_1",
    },
    {
      "is_recipe": false,
      "ingredient": "ingredient name",
      "id": "unique_id_2",
    },
    {
      ...
    },
  ],
  "links": [
    {
      "source": "unique_id_1",
      "target": "unique_id_2"
    },
    {
      ...
    }
  ]
}
```


In [ ]:
recipe_last_id = 0
ingredient_last_id = 0

ingredients_set = {}  # set of unique ingredients

for category in df["category"].unique():
    for subcategory in df[df["category"] == category]["subcategory"].unique():
        recipes_ingredients_per_subcategory = {
            recipe["recipe_name"]: recipe["ingredients"] for recipe in recipes_ingredients
            if recipe["category"] == category and recipe["subcategory"] == subcategory
        }

        ingredient_nodes = []
        ingredients_current_set = set()

        recipe_nodes = []

        recipe_links = []

        for i, (recipe_name, recipe_ingredients) in enumerate(recipes_ingredients_per_subcategory.items()):
            for ingredient in recipe_ingredients:
                if ingredient is None:
                    continue

                if ingredient not in ingredients_set:
                    # Assing unique id to each ingredient
                    ingredients_set[ingredient] = ingredient_last_id
                    ingredient_last_id += 1

                if ingredient not in ingredients_current_set:
                    ingredients_current_set.add(ingredient)

                    # Add ingredient node
                    ingredient_nodes.append(
                        {
                            "id": f"i{ingredients_set[ingredient]}",
                            "is_recipe": False,
                            "name": ingredient
                        }
                    )

                # Add links between recipe and ingredients
                recipe_links.append(
                    {
                        "source": f"r{recipe_last_id}",
                        "target": f"i{ingredients_set[ingredient]}"
                    }
                )

            # Add recipe node
            recipe_nodes.append(
                {
                    "id": f"r{recipe_last_id}",
                    "is_recipe": True,
                    "name": recipe_name
                }
            )

            recipe_last_id += 1

        # Create output directory
        os.makedirs(f"visualization/data/network/{category}", exist_ok=True)

        # Save to file
        with open(f"visualization/data/network/{category}/{subcategory}.json", "w", encoding="utf-8") as f:
            json.dump(
                {
                    "nodes": recipe_nodes + ingredient_nodes,
                    "links": recipe_links
                },
                f,
                ensure_ascii=False
            )

In [ ]:
# Create index file with paths to all subcategories
with open("visualization/data/network/index.json", "w", encoding="utf-8") as f:
    json.dump(
        {
            category: [
                subcategory for subcategory in df[df["category"] == category]["subcategory"].unique()
            ] for category in df["category"].unique()
        },
        f,
        ensure_ascii=False
    )